In [2]:
# Global normalisation based on calculated 2.5% and 97.5% percentile values calculated in Matlab
import arcpy
for x in range(0,6):
   if x == 0:
        Infile = "AriesRetention60Clipped.tif"
        OutfileNorm = r"D:\Data\Dropbox\Lactuca\Temp\1_AriesRetentionNormalised.tif"
        OutfileZeros = r"D:\Data\Dropbox\Lactuca\Temp\1_AriesRetentionZeros.tif"
        ValueMin = 0
        ValueMax = 3597.513175
   elif x == 1:
        Infile = "InVestRetention60Clipped.tif"
        OutfileNorm = r"D:\Data\Dropbox\Lactuca\Temp\2_InVestRetentionNormalised.tif"
        OutfileZeros = r"D:\Data\Dropbox\Lactuca\Temp\2_InVestRetentionZeros.tif"
        ValueMin = 0
        ValueMax = 19440.06
   elif x == 2:
        Infile = "TEEBErosionControl60Clipped.tif"
        OutfileNorm = r"D:\Data\Dropbox\Lactuca\Temp\3_TeebErosionControlNormalised.tif"
        OutfileZeros = r"D:\Data\Dropbox\Lactuca\Temp\3_TeebErosionControlZeros.tif"
        ValueMin = 0
        ValueMax = 337.45
   elif x == 3:
        Infile = "BeckyInversedDeposition60Clipped.tif"
        OutfileNorm = r"D:\Data\Dropbox\Lactuca\Temp\4_BeckyInvDeposNormalised.tif"
        OutfileZeros = r"D:\Data\Dropbox\Lactuca\Temp\4_BeckyInvDeposZeros.tif"
        ValueMin = 0.00001087197
        ValueMax = 1.258491128
   elif x == 4:
        Infile = "WWHillErosionInversed60.tif"
        OutfileNorm = r"D:\Data\Dropbox\Lactuca\Temp\5_WWHillErosionNormalised.tif"
        OutfileZeros = r"D:\Data\Dropbox\Lactuca\Temp\5_WWHillErosionZeros.tif"
        ValueMin = 0.9814061
        ValueMax = 0.0226619
   elif x == 5:
        Infile = "BCKErosion60Clipped.tif"
        OutfileNorm = r"D:\Data\Dropbox\Lactuca\Temp\Extra_BeckyErosion.tif"
        OutfileZeros = r"D:\Data\Dropbox\Lactuca\Temp\Extra_BeckyErosionZeros.tif"
        ValueMin = 0.00001568661
        ValueMax = 1.56882231339
   else:
        print("This code stops")
   with arcpy.EnvManager(overwriteOutput=True):
        out_raster = arcpy.sa.Minus(Infile, ValueMin); out_raster.save("TmpLayer1")
        out_raster = arcpy.sa.Con("TmpLayer1", 0, " TmpLayer1", "VALUE < 0")
        out_raster.save("TmpLayer2")
        out_raster = arcpy.sa.Divide("TmpLayer2", ValueMax); out_raster.save("TmpLayer3")
        out_raster = arcpy.sa.Con("TmpLayer3", 1, " TmpLayer3", "VALUE > 1")
        out_raster.save(OutfileNorm)
        out_raster = arcpy.sa.IsNull(OutfileNorm); out_raster.save("TmpLayer4")   
        out_raster = arcpy.sa.Con("TmpLayer4", "LandExtentasZeros60Degrees.tif", OutfileNorm,"Value = 1")
        out_raster.save(OutfileZeros)


In [ ]:
# Value extraction per Watershed, note as loop since sheds can overlap
# 'Type' distinguishes between value extraction or histograms (used for Gini calculations)
import arcpy
import numpy  
import csv  
arcpy.SetLogHistory(False)
store_buffs = "GemStatsShedsAllSorted" 
expression="IDs_DH  > -1"
with arcpy.da.SearchCursor(store_buffs, ["IDs_DH"], where_clause=expression) as cursor: 
  with arcpy.EnvManager(parallelProcessingFactor="100", cellSize = 0.001):
    for row in cursor:   
          # Name and select Shed
          exp = '"IDs_DH" = ' + str(row[0])  
          TestedShed = "shedsSHP_" + str(row[0]) 
          arcpy.analysis.Select(store_buffs, TestedShed, exp)
        # Incomplete list below for example purposes
          for x in range(0,3):
             if x == 0:
                 density_ras = "ModisForMasks.tif" 
                 OutBase = "D:/Data/Dropbox/Lactuca/Temp/shedsModis_"
                 Zonalrast = "ZonalModis"
                 Type = 2 # histogram!
             elif x == 1:
                 density_ras = "RuralUrbanClassesBinary.tif" 
                 OutBase = "D:/Data/Dropbox/Lactuca/Temp/shedsUrban_"
                 Zonalrast = "ZonalUrban"
                 Type = 1
             elif x == 2:
                 density_ras = "AriesRetentionNormalised.tif" 
                 OutBase = "D:/Data/Dropbox/Lactuca/Temp/shedsAries_"
                 Zonalrast = "ZonalAries"
                 Type = 1
             else:
                 print("This code stops")
             # Extract Data
             with arcpy.EnvManager(overwriteOutput=True):
                 if Type == 1:
                     #when used for extraction of statistics
                     arcpy.sa.ZonalStatisticsAsTable(TestedShed, "OBJECTID", density_ras, Zonalrast ,"DATA", "ALL")
                 elif Type == 2:                 
                     #when used as zonal histogram
                    arcpy.sa.ZonalHistogram(TestedShed, "IDs_DH", density_ras, Zonalrast, '', "ZONES_AS_FIELDS")
                 else:
                     print("This code stops")
                 # Write Data
                 table = Zonalrast
                 outfile = OutBase + str(row[0])+ ".csv"
                 fields = arcpy.ListFields(table)  
                 field_names = [field.name for field in fields]  
                 with open(outfile,'w') as f:  
                     w = csv.writer(f , delimiter = ',', lineterminator = '\n')  
                     #--write all field names to the output file  
                     w.writerow(field_names)  
                     for mow in arcpy.SearchCursor(table):  
                        field_vals = [mow.getValue(field.name) for field in fields]  
                        w.writerow(field_vals) 


In [1]:
# Cross reference extractions
import arcpy
import csv  
arcpy.SetLogHistory(False)
store_buffs = "GemStatsShedsAllSorted" 
expression="IDs_DH  > -1"
with arcpy.da.SearchCursor(store_buffs, ["IDs_DH"], where_clause=expression) as cursor: 
  with arcpy.EnvManager(parallelProcessingFactor="100", cellSize = 0.001):
    for row in cursor:   
          # Name and select Shed
          exp = '"IDs_DH" = ' + str(row[0])  
         # TestedShed = "shedsSHP_" + str(row[0]) 
          with arcpy.EnvManager(overwriteOutput=True):
              arcpy.analysis.Select(store_buffs, "TestedShed", exp)
          arcpy.analysis.Select(store_buffs, "TestedShed", exp)
         # Incomplete list below for example purposes
          for x in range(0,3):
             if x == 0:
                 density_ras = "MeanCarbon.tif" 
                 OutBase = "D:/Data/Dropbox/Lactuca/Temp/Carbon_Mean_"
                 Zonalrast = "Carbon_Mean"
                 Type = 1
             elif x == 1:
                 density_ras = "LeaevOneOutCarbon.tif" 
                 OutBase = "D:/Data/Dropbox/Lactuca/Temp/CarbonLeaveOne_"
                 Zonalrast = "Carbon_LeaveOne"
                 Type = 1
             elif x == 2:
                 density_ras = "Mean_RecreationEnsembleMorethan2models.tif" 
                 OutBase = "D:/Data/Dropbox/Lactuca/Temp/RecreationMean_"
                 Zonalrast = "Recreation_Mean"
                 Type = 1
             else:
                 print("This code stops")
             # Extract Data
             with arcpy.EnvManager(overwriteOutput=True):
                 if Type == 1:
                     #when used for extraction of statistics
                     arcpy.sa.ZonalStatisticsAsTable("TestedShed", "OBJECTID", density_ras, Zonalrast ,"DATA", "ALL")
                 elif Type == 2:                 
                     #when used as zonal histogram
                    arcpy.sa.ZonalHistogram("TestedShed", "IDs_DH", density_ras, Zonalrast, '', "ZONES_AS_FIELDS")
                 else:
                     print("This code stops")
                 # Write Data
                 table = Zonalrast
                 arcpy.management.AddField(table, "PlotID", "LONG", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')
                 arcpy.management.CalculateField(table, "PlotID", row[0], "PYTHON3", '', "LONG")
                 outfile = OutBase + str(row[0])+ ".csv"
                 fields = arcpy.ListFields(table)  
                 field_names = [field.name for field in fields]  
                 with open(outfile,'w') as f:  
                     w = csv.writer(f , delimiter = ',', lineterminator = '\n')  
                     #--write all field names to the output file  
                     w.writerow(field_names)  
                     for mow in arcpy.SearchCursor(table):  
                        field_vals = [mow.getValue(field.name) for field in fields]  
                        w.writerow(field_vals)


In [ ]:
# ArcMap codes for delineating watersheds
import arcpy
for x in range(1,2):
   if x == 0:
        PointsNames = "NA_west_PointTable_Snapped"
        AccRaster = " FlowAcc_NorthAmericaWest"
        DirRaster = "FlowDir_NorthAmericaWest"
        expression= "ID_DH > 1346"
   elif x == 1:
        PointsNames = "Europe_PointTable_Snapped"
        AccRaster = " FlowAcc_Europe"
        DirRaster = "FlowDir_Europe"
        expression= "ID_DH > -1"
   else:
        print("This code stops")
   with arcpy.da.SearchCursor(PointsNames, ["ID_DH"], where_clause=expression) as cursor: 
       for row in cursor:
          with arcpy.EnvManager(overwriteOutput=True):
              exp = '"ID_DH" = ' + str(row[0])
              arcpy.analysis.Select(PointsNames, "SelectedPoint", exp)
              out_raster = arcpy.sa.SnapPourPoint("SelectedPoint", AccRaster, 0.008333, "OBJECTID"); out_raster.save("SnapSelected")
              # So it is allowed 1-km off, note it need to be pre-snapped with the near tool
              out_raster = arcpy.sa.Watershed(DirRaster, "SnapSelected", "Value"); out_raster.save("Watersh_Rast")
              with arcpy.EnvManager(outputZFlag="Disabled", outputMFlag="Disabled"):
               arcpy.conversion.RasterToPolygon("Watersh_Rast", "RastPoly", "NO_SIMPLIFY", "Value", "SINGLE_OUTER_PART", None)
              arcpy.management.AddField("RastPoly", "IDs_DH", "LONG", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')
              arcpy.management.CalculateField("RastPoly", "IDs_DH", row[0], "PYTHON3", '', "TEXT")
              arcpy.management.JoinField("RastPoly", "IDs_DH", "SelectedPoint", "ID_DH", "MeanValue")
              outfile = "D:/Data/Dropbox/Lactuca/temp/Shed_" + str(row[0])
              arcpy.management.CopyFeatures("RastPoly",outfile, '', None, None, None)

In [ ]:
## MATLAB example codes for combining outputed csv files
List = dataset(1,'VarNames',{'Basin_DH'});
List.Area = 1;
List.Number = 1;
# Copy the data into those Lists
for i = 1:length(WaterSheds.Basin_ID)
    nrm = WaterSheds.Basin_ID(i);
   fndr = find(List.Basin_DH ==nrm);
   AriesRentention(i,1) = WaterSheds.Basin_ID(i);
   if isempty(fndr) ~= 1
       AriesRentention(i,2) = List.Number(fndr);
       AriesRentention(i,3) =  List.Area(fndr);
   else
       AriesRentention(i,2) = NaN;
       AriesRentention(i,3) =  NaN;
   end   
end
clear nrm fndr i
save('AriesRentention','AriesRentention') 
clear List
List = dataset(1,'VarNames',{'Basin_DH'});
List.Area = 1;
List.Number = 1;